## **EXERCÍCIO 09 - RAG**

## **CESAR SCHOOL**
* Pós-graduação em Engenharia e Análise de Dados - 2023.2
* **Disciplina: Tópicos Complementares**
* Professor: **Silvan Ferreira**
* Aluno: **Allan Bispo** - apsb@cesar.school

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
!pip install lambdata langchain_community langchain_openai langchain_text_splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141

## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [3]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("mw-body-content"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://pt.wikipedia.org/wiki/Centro_de_Estudos_e_Sistemas_Avan%C3%A7ados_do_Recife",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

6915

In [4]:
print(docs[0].page_content[500:1000])

cife, também conhecido por seu acrônimo CESAR, é um centro de pesquisa e inovação sem fins lucrativos com sede na cidade do Recife, Pernambuco e filiais em Sorocaba, Curitiba e Manaus.[1] O CESAR foi fundado em 1996 por três professores do Centro de Informática da UFPE, Silvio Meira, Fábio Silva e Ismar Kaufman, como forma de aproximar a academia do mercado.[2]
Em 2019, o centro conta com mais de 600 funcionários e em 2018 seu faturamento foi da ordem de R$ 100 milhões.[1][3][4][5]
O CESAR é par


### Dividindo Documentos - Splitting/Chunking

In [7]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

10

In [ ]:
print(all_splits[2].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Indexando - Store

In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.3 MB/s eta 0:00:00


In [8]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [9]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Onde está localizado o CESAR?")

len(retrieved_docs)

6

In [10]:
print(retrieved_docs[0].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Buscando e Recuperando Informações - Retrieve

In [11]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [12]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n')]


### Gerando Respostas - Generate

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [15]:
for chunk in rag_chain.stream("Onde fica o CESAR?"):
    print(chunk, end="", flush=True)

O CESAR fica em Recife, Pernambuco.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [ ]:
# Para informação: abaixo o conteúdo do arquivo 'rag_texto.txt'
# O script RAG vai ler o arquivo e responder a um loop de perguntas com base no conteúdo do mesmo.

"""
Em uma nebulosa distante, uma espada antiga repousava, irradiando alegria.
Em Xangai, sob a aurora boreal, um evento absurdo e eterno ocorria.
Plutão, sempre misterioso, existia em um estado de talvez.
Acima de tudo, uma luz violeta começava a brilhar, desafiando a escuridão.
Nunca antes se viu algo assim.
Questionar a realidade era inevitável, pois o que parecia impossível agora existia diante dos olhos de todos.
A beleza do cosmos revelava segredos ocultos, e a humanidade, maravilhada, contemplava o infinito.
Cada estrela, cada planeta, contava uma história de esperança e descoberta.
"""

'\nEm uma nebulosa distante, uma espada antiga repousava, irradiando alegria.\nEm Xangai, sob a aurora boreal, um evento absurdo e eterno ocorria.\nPlutão, sempre misterioso, existia em um estado de talvez.\nAcima de tudo, uma luz violeta começava a brilhar, desafiando a escuridão.\nNunca antes se viu algo assim.\nQuestionar a realidade era inevitável, pois o que parecia impossível agora existia diante dos olhos de todos.\nA beleza do cosmos revelava segredos ocultos, e a humanidade, maravilhada, contemplava o infinito.\nCada estrela, cada planeta, contava uma história de esperança e descoberta.\n'

In [ ]:
# Importando a biblioteca de carregamento de documentos da LangChain
from langchain_community.document_loaders import TextLoader

In [ ]:
# Lendo o arquivo de texto local
file_path = "/content/rag_texto.txt"
loader = TextLoader(file_path)
docs = loader.load()

In [ ]:
# Configurando o chunking para dividir o documento em pedaços
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True)
all_splits = text_splitter.split_documents(docs)

In [ ]:
# Configurando o recuperador de documentos usando similaridade
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("O que representa a luz violeta que começa a brilhar?")

In [ ]:
# Definindo o template do prompt para o assistente
system_template = """Você é um assistente para tarefas de perguntas e respostas sobre o cosmos e fenômenos misteriosos. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases, mantenha a resposta concisa e poética, e fale apenas o necessário.
Pergunta: {question}
Contexto: {context}
Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
# Função para formatar os documentos recuperados
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Configurando a cadeia de RAG
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser())

In [ ]:
# Executando a cadeia de RAG com uma pergunta exemplo
for chunk in rag_chain.stream("O que representa a luz violeta que começa a brilhar?"):
    print(chunk, end="", flush=True)

A luz violeta que começa a brilhar simboliza a revelação de segredos ocultos no cosmos, desafiando a escuridão e a realidade. É um convite à esperança e à descoberta, onde o impossível se torna visível.

In [ ]:
lista_perguntas = [
    "O que representa a luz violeta que começa a brilhar?",
    "Qual é o significado da espada antiga que repousa na nebulosa distante?",
    "O que acontece durante o evento absurdo e eterno em Xangai sob a aurora boreal?",
    "Como Plutão pode existir em um estado de 'talvez'?",
    "O que os segredos ocultos do cosmos revelam sobre a beleza do universo?"
]

In [ ]:
# Iterando sobre cada pergunta e executando a cadeia de RAG para cada uma
for i, pergunta in enumerate(lista_perguntas):
    print(f"Pergunta {i+1}: {pergunta}")
    for chunk in rag_chain.stream(pergunta):
        print(chunk, end="", flush=True)
    print("\n")

Pergunta 1: O que representa a luz violeta que começa a brilhar?
A luz violeta que começa a brilhar representa a esperança e os segredos do cosmos, desafiando a escuridão e revelando o impossível diante dos olhos da humanidade. É um convite à contemplação do infinito e das histórias ocultas nas estrelas.

Pergunta 2: Qual é o significado da espada antiga que repousa na nebulosa distante?
A espada antiga na nebulosa é um símbolo de alegria e mistério, refletindo as histórias de esperança que o cosmos guarda. Ela desafia a escuridão, revelando segredos e maravilhas que nos convidam a contemplar o infinito.

Pergunta 3: O que acontece durante o evento absurdo e eterno em Xangai sob a aurora boreal?
Sob a aurora boreal em Xangai, um evento absurdo e eterno desafiava a realidade, revelando uma luz violeta que iluminava o impossível. A humanidade, maravilhada, contemplava os segredos do cosmos, onde cada estrela sussurrava histórias de esperança.

Pergunta 4: Como Plutão pode existir em um e

####RESPOSTA:
* Talvez devido ao fato do texto no arquivo ser totalmente desconexo, criado dessa forma para tentar que a LLM não conhecesse previamente respostas sobre o mesmo, as respostas do modelo RAG quase beiram a alucinação.
* Mas parece mesmo é que o modelo conseguiu identificar os sentimentos no texto por via da similaridade com as palavras das perguntas, gerando respostas coerentes com o conteúdo do arquivo.